## Obtain Mask Dataframes- this time, with better pos/neg classification

#### Import Libraries

In [1]:
import ast
import pandas as pd
import re

#### Start off with processed_geotweets_df_USA

In [2]:
processed_geotweets_df_USA = pd.read_csv("./Stored_Dataframes/processed_geotweet_df_USA.csv", lineterminator='\n')
processed_geotweets_df_USA.head(10) 

,possibly_sensitive,entities,text,public_metrics,tweet_id,context_annotations,lang,in_reply_to_user_id,author_id,source,...,referenced_tweet_ids,users_mentioned,place_id,latitude,longitude,day,full_name,state,code,week\r
0,False,NaN,fuck corona i miss my boyfriend,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1240727871095631872,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,859271883279159297,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
1,False,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",@realDonaldTrump is a POS! https://t.co/3AswTG...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240741265064341505,"[{'domain': {'id': '10', 'name': 'Person', 'de...",en,2.507388e+07,781285034703491073,Twitter for iPhone,...,['1240701140141879298'],['realDonaldTrump'],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
2,False,"{'annotations': [{'start': 57, 'end': 67, 'pro...",You know what we all need during this Corona o...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240745322952790016,"[{'domain': {'id': '10', 'name': 'Person', 'de...",en,NaN,2841636924,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
3,False,NaN,This corona virus ain’t got shit over the vall...,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1240753298140872705,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,888083561743384576,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
4,False,"{'urls': [{'start': 12, 'end': 35, 'url': 'htt...",Corona who? https://t.co/VRJuZvGoaV,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240759511423397889,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,2996168750,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
5,False,"{'annotations': [{'start': 25, 'end': 30, 'pro...",They think my sister has Corona 👀,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240761334205014016,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,3517880953,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
6,False,"{'annotations': [{'start': 31, 'end': 37, 'pro...","We know 1st-hand, hospitals in Detroit testing...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240763146597351424,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,1002264486890299392,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
7,False,"{'annotations': [{'start': 88, 'end': 93, 'pro...",Corona actually *this* close to making me watc...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240768630549172224,"[{'domain': {'id': '46', 'name': 'Brand Catego...",en,NaN,19273823,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
8,False,"{'mentions': [{'start': 0, 'end': 7, 'username...",@heIIIa Corona Cove,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240770507273695232,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,8.872029e+17,1489852370,Twitter for iPhone,...,['1240765035900620800'],['HeiiiA'],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0
9,False,"{'annotations': [{'start': 33, 'end': 43, 'pro...",Two cases of the corona virus in Bakersfield. ...,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1240778790332026880,"[{'domain': {'id': '123', 'name': 'Ongoing New...",en,NaN,211352793,Twitter for iPhone,...,[],[],fbd6d2f5a4e4a15e,37.269176,-119.306607,1,"California, USA",California,CA,0


#### Create mask df

In [3]:
mask_df = processed_geotweets_df_USA.copy()
mask_df = mask_df[['text', 'public_metrics', 'state', 'code', 'author_id', 'week\r']]
mask_df = mask_df.rename(columns={'week\r': 'week'})
def get_retweet_count(public_metrics):
    d = ast.literal_eval(public_metrics)
    return d['retweet_count']
def get_reply_count(public_metrics):
    d = ast.literal_eval(public_metrics)
    return d['reply_count']
def get_like_count(public_metrics):
    d = ast.literal_eval(public_metrics)
    return d['like_count']
def get_quote_count(public_metrics):
    d = ast.literal_eval(public_metrics)
    return d['quote_count']
regexpmask = re.compile(r'(?i)mask') 
def maskrelated(text):
    if regexpmask.search(text):
        return 1
    return 0
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    #text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)
    return text
def extract_hash_tags(s):
    return set(part[1:].lower() for part in s.split() if part.startswith('#'))
def extract_mask_hashtags(hashtag_set):
    h_set = set()
    for word in hashtag_set:
        if "mask" in word:
            h_set.add(word)
    return h_set
def mask_in_hashtags(hashtag_set):
    return hashtag_set != set()
mask_df['retweet_count'] = mask_df['public_metrics'].apply(get_retweet_count)
mask_df['reply_count'] = mask_df['public_metrics'].apply(get_reply_count)
mask_df['like_count'] = mask_df['public_metrics'].apply(get_like_count)
mask_df['quote_count'] = mask_df['public_metrics'].apply(get_quote_count)
mask_df['mask_in_text'] = mask_df['text'].apply(maskrelated)
mask_df['clean_text'] = mask_df['text'].apply(lambda x: clean_tweets(x))
mask_df['hashtags'] = mask_df['text'].apply(extract_hash_tags)
mask_df['mask_hashtags'] = mask_df['hashtags'].apply(extract_mask_hashtags)
mask_df['mask_in_hashtags'] = mask_df['mask_hashtags'].apply(mask_in_hashtags)

In [4]:
!pip install -U liwc
import re
from collections import Counter

def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

import liwc
parse, category_names = liwc.load_token_parser('LIWC2007_English080730.dic')

def liwc_analysis(twt):
    twt_tokens = tokenize(twt)
    # now flatmap over all the categories in all of the tokens using a generator:
    twt_counts = Counter(category for token in twt_tokens for category in parse(token))
    # and print the results:
    return dict(twt_counts)
mask_df['liwc_analysis'] = mask_df['clean_text'].apply(liwc_analysis)

In [5]:
# to make better, only adjectives
# !pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']
def posneg(polarity):
    if polarity > 0:
        return "positive"
    elif polarity < 0:
        return "negative"
    return "neutral"
mask_df['polarity'] = mask_df['clean_text'].apply(sentiment_analyzer_scores)
mask_df['posneg_vader']=mask_df['polarity'].apply(posneg)

In [6]:
maskintext_df = mask_df[mask_df['mask_in_text']==1]
maskinhashtag_df = mask_df[mask_df['mask_in_hashtags']==1]
postwts_vader_df = maskinhashtag_df[maskinhashtag_df['posneg_vader']=='positive'][['author_id', 'text', 'polarity', 'liwc_analysis', 'retweet_count', 'like_count', 'quote_count', 'reply_count', 'code', 'week']].reset_index() 
negtwts_vader_df = maskinhashtag_df[maskinhashtag_df['posneg_vader']=='negative'][['author_id', 'text', 'polarity', 'liwc_analysis', 'retweet_count', 'like_count', 'quote_count', 'reply_count', 'code', 'week']].reset_index()

In [7]:
pos = set()
neg = set()
with open('maskhashtags.txt') as hashtagfile:
    for line in hashtagfile:
        contents = line.strip("\n").split(" ")
        if (len(contents) > 1):
            if contents[1] == 'p':
                pos.add(contents[0].lower())
            elif contents[1] == 'n':
                neg.add(contents[0].lower())
neg.remove("maskoff")
neg.remove("nomasks")
def getposhashtagcount(hset):
    return len(hset.intersection(pos))
def getneghashtagcount(hset):
    return len(hset.intersection(neg))
def posneghashtag(row):
    if row['poshashtag_count'] > row['neghashtag_count']:
        return 'positive'
    elif row['poshashtag_count'] < row['neghashtag_count']:
        return 'negative'
    else:
        return 'neutral'
maskinhashtag_df['poshashtag_count'] = maskinhashtag_df['mask_hashtags'].apply(getposhashtagcount)
maskinhashtag_df['neghashtag_count'] = maskinhashtag_df['mask_hashtags'].apply(getneghashtagcount)
maskinhashtag_df['posneg_hashtag'] = maskinhashtag_df.apply(posneghashtag, axis=1)
# To deal with mislabels, these will override
defpos = set(["shitisreal", "covidiot", "covididiot", "covidiots", "covididiots"])
defneg = set(["covidsets"])
defneutral = set(["customerservice"])
regexppos = re.compile(r'(?i)please wear|still have covid, right?|AVOID @united flights|No masks. No distancing. Shame.|Great. I live in #NoMaskville. Damn idiots|It angers me that they have taken things so lightly|no masks on how can this be safe?|where they\'ll spread COVID19  next?|act as #volunteers to #help') 
regexpneg = re.compile(r'(?i)Grow a set|I will not wear a mask|ridiculous mask policy')
def override(row):
    if defpos.intersection(row['hashtags']) != set():
        return "positive"
    if regexppos.search(row['text']):
        return "positive"
    if defneg.intersection(row['hashtags']) != set():
        return "negative"
    if regexpneg.search(row['text']):
        return "negative"
    if defneutral.intersection(row['hashtags']) != set():
        return "neutral"
    return row['posneg_hashtag']
maskinhashtag_df['posneg_hashtag'] = maskinhashtag_df.apply(override, axis=1)
postwts_hashtag_df = maskinhashtag_df[maskinhashtag_df['posneg_hashtag']=='positive'][['author_id', 'text', 'polarity', 'liwc_analysis', 'retweet_count', 'like_count', 'quote_count', 'reply_count', 'code', 'week', 'hashtags']].reset_index() 
negtwts_hashtag_df = maskinhashtag_df[maskinhashtag_df['posneg_hashtag']=='negative'][['author_id', 'text', 'polarity', 'liwc_analysis', 'retweet_count', 'like_count', 'quote_count', 'reply_count', 'code', 'week', 'hashtags']].reset_index()

C:\Users\hshma\anaconda3\envs\D3M\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
import collections
s = []
for _, row in maskinhashtag_df.iterrows():
    s.extend(list(row['mask_hashtags']))
def countFrequency(my_list):
    poshashtag_freqs = {}
    neghashtag_freqs = {}
    for item in my_list:
        if item in poshashtag_freqs:
            poshashtag_freqs[item] += 1
        elif item in neghashtag_freqs:
            neghashtag_freqs[item] += 1
        elif item in pos:
            poshashtag_freqs[item] = 1
        elif item in neg:
            neghashtag_freqs[item] = 1
    return poshashtag_freqs, neghashtag_freqs
positive_mask_hashtag_freqs, negative_mask_hashtag_freqs = countFrequency(s)
positive_mask_hashtag_freqs = dict(sorted(positive_mask_hashtag_freqs.items(), key=lambda item: item[1], reverse=True))
negative_mask_hashtag_freqs = dict(sorted(negative_mask_hashtag_freqs.items(), key=lambda item: item[1], reverse=True))

#### We need more data clearly

In [ ]:
print("Total Number of positive tweets: {}".format(postwts_hashtag_df.shape[0]))
print("Total Number of negative tweets: {}".format(negtwts_hashtag_df.shape[0]))

#### Just to see how good we classify

In [ ]:
for _, row in postwts_hashtag_df[:100].iterrows():
    print("\nNEW TWEET")
    print(row['text'])

In [ ]:
for _, row in negtwts_hashtag_df.iterrows():
    print("\nNEW TWEET")
    print(row['text'])
    print(row['hashtags'])

#### Just for testing

In [ ]:
for _, row in maskinhashtag_df.iterrows():
    if 'mandatemasks??' in list(row['mask_hashtags']):
        print(row['text'])

In [ ]:
mask_df.to_csv('./Stored_Dataframes/MaskDFs/mask_df.csv', sep=",", line_terminator='\n', encoding='utf-8')
maskintext_df.to_csv('./Stored_Dataframes/MaskDFs/maskintext_df.csv', sep=",", line_terminator='\n', encoding='utf-8') 
maskinhashtag_df.to_csv('./Stored_Dataframes/MaskDFs/maskinhashtag_df.csv', sep=",", line_terminator='\n', encoding='utf-8')
postwts_vader_df.to_csv('./Stored_Dataframes/MaskDFs/postwts_vader_df.csv', sep=",", line_terminator='\n', encoding='utf-8')
negtwts_vader_df.to_csv('./Stored_Dataframes/MaskDFs/negtwts_vader_df.csv', sep=",", line_terminator='\n', encoding='utf-8')
postwts_hashtag_df.to_csv('./Stored_Dataframes/MaskDFs/postwts_hashtag_df.csv', sep=",", line_terminator='\n', encoding='utf-8')
negtwts_hashtag_df.to_csv('./Stored_Dataframes/MaskDFs/negtwts_hashtag_df.csv', sep=",", line_terminator='\n', encoding='utf-8')